# Initial testing for creating KML files to visualize total columns

In [3]:
#Import Packages
import pandas as pd
import simplekml
import os
import xarray as xr
import numpy as np
import sys
sys.path.append('..')
import funcs.ac_funcs as ac

In [4]:
base_project_dir = '/Users/agmeyer4/LAIR_1'
kml_save_dir = '/Users/agmeyer4/Google Drive/My Drive/Documents/Work/Solicitations/DoE_Methane_3b/Figures/earth_model/uinta_basin'

# Grid

In [141]:
# Create the grid
ll_spacing = 0.2
z_spacing = 5000

grid_extent = {'lon_min':-110.6,
                  'lon_max':-109,
                  'lat_min':39.7,
                  'lat_max':40.7} 

# Define the latitude, longitude, and elevation arrays
lats = np.arange(grid_extent['lat_min'],grid_extent['lat_max'], ll_spacing)
lons = np.arange(grid_extent['lon_min'],grid_extent['lon_max'], ll_spacing)
zasls = np.arange(1000, 20000, z_spacing)  # Elevation levels from 1000 to 5000 in steps of 500

# Initialize a list to store the line segments
air_grid_segments = []

# Create vertical and horizontal line segments
for l in lats:
    for ln in lons:
        for zasl in zasls:
            # Horizontal segments (latitude changes)
            if l < lats[-1]:
                air_grid_segments.append({'coords':((ln, l, zasl), (ln, l + ll_spacing, zasl)),'zasl':zasl})
            # Vertical segments (longitude changes)
            if ln < lons[-1]:
                air_grid_segments.append({'coords':((ln, l, zasl), (ln+ll_spacing, l, zasl)),'zasl':zasl})
            # Elevation segments (elevation changes)
            if zasl < zasls[-1]:
                air_grid_segments.append({'coords':((ln, l, zasl), (ln, l, zasl+z_spacing)),'zasl':zasl})

ground_segments = []
for l in lats:
    for ln in lons:
        if l < lats[-1]:
            ground_segments.append(((ln, l, 0), (ln, l + ll_spacing, 0)))
        # Vertical segments (longitude changes)
        if ln < lons[-1]:
            ground_segments.append(((ln, l, 0), (ln + ll_spacing, l, 0)))


In [142]:
#Put the grid in the kml
name = 'grid'

kml_name = f'{name}.kml' #storage

kml = simplekml.Kml() #initialize the kml file
fol = kml.newfolder(name='air_grid')
zfols = {}
for zasl in zasls:
    zfols[zasl] = fol.newfolder(name=zasl)

for segment in air_grid_segments:
    zfol = zfols[segment['zasl']]
    ls = zfol.newlinestring(name=segment['zasl'],
                            coords=[segment['coords'][0],segment['coords'][1]],                           
                            altitudemode='absolute')

fol = kml.newfolder(name='ground_grid')
for segment in ground_segments:
    ls = fol.newlinestring(name=f'grid_ls',
                            coords=[segment[0],segment[1]],                           
                            altitudemode='relativeToGround')
    

kml.save(os.path.join(kml_save_dir,kml_name))

# Ground based solar

In [31]:
# Running this will load create the DEM handler 
dem_folder = os.path.join(base_project_dir,'Data/DEM/')
dem_fname = 'ASTGTM_NC.003_30m_aid0001.nc'
dem_typeid = 'aster'
my_dem_handler = ac.DEM_handler(dem_folder,dem_fname,dem_typeid) #define the handler




In [75]:
ll_spacing = 0.2

grid_extent = {'lon_min':-110.3,
                  'lon_max':-109.299,
                  'lat_min':39.8,
                  'lat_max':40.4} 

# Define the latitude, longitude, and elevation arrays
lats = np.arange(grid_extent['lat_min'],grid_extent['lat_max'], ll_spacing)
lons = np.arange(grid_extent['lon_min'],grid_extent['lon_max'], ll_spacing)

inst_locs = []
for lat in lats:
    for lon in lons:
        inst_locs.append({'lat':lat,'lon':lon,'zasl':my_dem_handler.get_nearest_elev(lat,lon)})


In [76]:
#Instrument defs
# instruments = {'WBB':{'lat':40.766,'lon':-111.847,'zasl':1470},
#                'DBK':{'lat':40.538,'lon':-112.0696,'zasl':1584},
#                'T1':{'lat':40.4697,'lon':-111.926,'zasl':1389},
#                'T2':{'lat':40.549,'lon':-111.807,'zasl':1559},
#                'T3':{'lat':40.640,'lon':-111.805,'zasl':1421},
#                'T4':{'lat':40.700,'lon':-111.935,'zasl':1300},
#                'T5':{'lat':40.661,'lon':-112.0736,'zasl':1502},
#                'T6':{'lat':40.743,'lon':-112.177,'zasl':1288},
#                'T7':{'lat':40.825,'lon':-112.005,'zasl': 1286},
#                'T8':{'lat':40.855,'lon':-111.858,'zasl':1654}
#                }


In [77]:
interval = '1H'

#Next define the datetime range
dt1_str = '2022-10-19 14:00:00'
dt2_str = '2022-10-20 04:00:00' #start datetime
timezone = 'UTC' #this should nearly always be UTC, but can identify if your datetime strings above are in a different timezone

#Now format the datetime and timezone into useful formats
dt1 = ac.dtstr_to_dttz(dt1_str,timezone)
dt2 = ac.dtstr_to_dttz(dt2_str,timezone)

#Define the heights above the instrument (in m) from which we would like to release particles
z_ail_list = [0,15000]

In [78]:
slant_dfs = {}
i = 0
for inst in inst_locs:
    gsh = ac.ground_slant_handler(inst['lat'],inst['lon'],inst['zasl'],z_ail_list)
    slant_df = gsh.run_slant_at_intervals(dt1,dt2,my_dem_handler,interval=interval)
    slant_df = slant_df.dropna()
    slant_dfs[i] = slant_df
    i+=1

Adding receptor lat/lons along the slant column
Adding surface height and receptor elevation above ground level
adding surface and agl heights


/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


Adding receptor lat/lons along the slant column
Adding surface height and receptor elevation above ground level
adding surface and agl heights


/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


Adding receptor lat/lons along the slant column
Adding surface height and receptor elevation above ground level
adding surface and agl heights


...

/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


Adding receptor lat/lons along the slant column
Adding surface height and receptor elevation above ground level
adding surface and agl heights


/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


Adding receptor lat/lons along the slant column
Adding surface height and receptor elevation above ground level
adding surface and agl heights


/Users/agmeyer4/opt/anaconda3/envs/atmos_column/lib/python3.11/site-packages/pysolar/solartime.py:111: UserWarning: I don't know about leap seconds after 2021
  warnings.warn \


In [79]:
name = 'column_network_1hr'

kml_name = f'{name}.kml' #storage
icon_size = 0.6 #how big we want the receptor points to be 

kml = simplekml.Kml() #initialize the kml file

#Add the slant columns
for inst_id,slant_df in slant_dfs.items():
    fol = kml.newfolder(name=inst_id)
    for dt,newdf in slant_df.groupby(level=0): #groupby the first index -- datetime
        time_str = dt.strftime('%H:%M %Z') #grab the datetime string for labeling
        plotdf = newdf.droplevel(0).reset_index() #get the new dataframe at that datetime level and reset the index so z_asl is a column not an index

        slant_id = time_str #the slant ID is best associated with the time string 
        
        #the linestring is just a path between the first and last points of the column receptor, to visualize the column
        ls = fol.newlinestring(name=f'{slant_id}_ls',coords=[(plotdf.iloc[0]['receptor_lon'],plotdf.iloc[0]['receptor_lat'],plotdf.iloc[0]['receptor_zasl'])
                                                            ,(plotdf.iloc[-1]['receptor_lon'],plotdf.iloc[-1]['receptor_lat'],plotdf.iloc[-1]['receptor_zasl'])],
                                                            altitudemode='absolute')
        ls.timespan.begin = dt.strftime('%Y-%m-%dT%H:%M:%S')
        end = dt + pd.to_timedelta(interval)
        ls.timespan.end = end.strftime('%Y-%m-%dT%H:%M:%S')
        ls.style.linestyle.width = 2.8
        #ls.style.linestyle.color = 'f5f242'

kml.save(os.path.join(kml_save_dir,kml_name))

# Draped Images

In [85]:
name = 'draped_ch4_inventory'


kml_name = f'{name}.kml' #storage
icon_size = 0.6 #how big we want the receptor points to be 

kml = simplekml.Kml() #initialize the kml file



raster_extent= {'lon_min':-110.6,
                  'lon_max':-109.201,
                  'lat_min':39.7,
                  'lat_max':40.5}

name = 'inventory'
bbox = raster_extent
raster_path = '/Users/agmeyer4/Library/CloudStorage/GoogleDrive-agmeyer4@gmail.com/My Drive/Documents/Work/Solicitations/DoE_Methane_3b/Figures/earth_model/uinta_basin/uinta_ch4_EPA.jpg'
#Set the bounding box with lat lons for our png file
numpy_bounds = [ (raster_extent['lon_min'],raster_extent['lat_min']),
                 (raster_extent['lon_max'],raster_extent['lat_min']),
                 (raster_extent['lon_max'],raster_extent['lat_max']),
                 (raster_extent['lon_min'],raster_extent['lat_max']) ] 
corner_coords = [ (float(x), float(y)) for x,y in numpy_bounds ]

doc = kml.newdocument(name = 'EPA')
#Add ground overlay
ground = doc.newgroundoverlay(name=name, color='ffffffff')
ground.gxlatlonquad.coords = corner_coords
ground.icon.href = raster_path

kml.save(os.path.join(kml_save_dir,kml_name))

In [86]:

name = 'footprint'

kml_name = f'{name}.kml' #storage
icon_size = 0.6 #how big we want the receptor points to be 

kml = simplekml.Kml() #initialize the kml file



extent=[-112.3,-111.57,40.39,40.95]

name = 'footprint'
bbox = extent
raster_path = '/Users/agmeyer4/LAIR_1/Figures/stilt_footprint_pngs/202210191700_-111.842_40.7613_384.86.png'

#Set the bounding box with lat lons for our png file
numpy_bounds = [ (bbox[0],bbox[2]),(bbox[1],bbox[2]),(bbox[1],bbox[3]),(bbox[0],bbox[3]) ]
corner_coords = [ (float(x), float(y)) for x,y in numpy_bounds ]


doc = kml.newdocument(name = 'footprint')
#Add ground overlay
ground = doc.newgroundoverlay(name=name, color='ffffffff')
ground.gxlatlonquad.coords = corner_coords
ground.icon.href = raster_path

kml.save(os.path.join(kml_save_dir,kml_name))

# Satellite

In [137]:
ll_spacing = 0.2
z_sat = 60000
grid_extent = {'lon_min':-109.8,
                  'lon_max':-109.6,
                  'lat_min':39.7,
                  'lat_max':40.5} 

# Define the latitude, longitude, and elevation arrays
lats = np.arange(grid_extent['lat_min'],grid_extent['lat_max'], ll_spacing)
lon_min = -109.6

In [138]:
#Put the grid in the kml
name = 'satellite'

kml_name = f'{name}.kml' #storage

kml = simplekml.Kml() #initialize the kml file

i = 0
for l in lats:
    fol = kml.newfolder(name=i)
    sat_point = {'lat':l+ll_spacing/2,'lon':lon_min+ll_spacing/2}
    
    ls = fol.newlinestring(name=f'grid_ls',
                    coords=((lon_min, l, 0), (lon_min, l + ll_spacing, 0)),                           
                    altitudemode='relativeToGround')
    ls = fol.newlinestring(name=f'grid_ls',
                    coords=((lon_min+ ll_spacing, l, 0), (lon_min, l , 0)),                           
                    altitudemode='relativeToGround')
    ls = fol.newlinestring(name=f'grid_ls',
                    coords=((lon_min, l+ ll_spacing, 0), (lon_min+ ll_spacing, l+ ll_spacing , 0)),                           
                    altitudemode='relativeToGround')
    ls = fol.newlinestring(name=f'grid_ls',
                    coords=((lon_min+ ll_spacing, l+ ll_spacing, 0), (lon_min+ ll_spacing, l , 0)),                           
                    altitudemode='relativeToGround')
    ls = fol.newlinestring(name=f'grid_ls',
                coords=((lon_min, l, 0), (sat_point['lon'], sat_point['lat'] , z_sat)),                           
                altitudemode='relativeToGround')
    ls = fol.newlinestring(name=f'grid_ls',
                coords=((lon_min+ll_spacing, l, 0), (sat_point['lon'], sat_point['lat'] , z_sat)),                           
                altitudemode='relativeToGround')
    ls = fol.newlinestring(name=f'grid_ls',
                coords=((lon_min+ll_spacing, l+ll_spacing, 0), (sat_point['lon'], sat_point['lat'] , z_sat)),                           
                altitudemode='relativeToGround')
    ls = fol.newlinestring(name=f'grid_ls',
                coords=((lon_min, l+ll_spacing, 0), (sat_point['lon'], sat_point['lat'] , z_sat)),                           
                altitudemode='relativeToGround')
    pt = fol.newpoint(name='sat_pt',coords = [(sat_point['lon'],sat_point['lat'],z_sat)],
                      altitudemode = 'relativeToGround')
    i+=1

kml.save(os.path.join(kml_save_dir,kml_name))